In [1]:
%matplotlib inline
import datetime
import time
import numpy as np
import pandas as pd
import pickle


Bad key "ytick.alignment" on line 268 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

Bad key "xtick.alignment" on line 250 in
/usr/share/matplotlib/mpl-data/stylelib/classic.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def adjust_id(df_element):
    adj = df_element.replace('email','').replace('ip','').replace('card','')
    return int(float(adj))

def string_to_timestamp(df_row):
    time_stamp = time.strptime(df_row.creationdate, '%Y-%m-%d %H:%M:%S')
    return time.mktime(time_stamp)

def adjust_creationdate(df_row):
    year_info = datetime.datetime.strptime(df_row.creationdate,'%Y-%m-%d %H:%M:%S').year
    month_info = datetime.datetime.strptime(df_row.creationdate,'%Y-%m-%d %H:%M:%S').month
    day_info = datetime.datetime.strptime(df_row.creationdate,'%Y-%m-%d %H:%M:%S').day
    creationdate = str(year_info)+'-'+str(month_info)+'-'+str(day_info)#Date of transaction 
    return creationdate

In [3]:
src = 'data_for_student_case.csv'
df = pd.read_csv(src)


df = df[(df['simple_journal'] != 'Refused') & ~(df['mail_id'].str.contains("na", case=False)) & ~(df['bin'].isnull())]
adjust_id_cols = ['mail_id', 'ip_id', 'card_id']
df[adjust_id_cols] = df[adjust_id_cols].applymap(adjust_id)

df['cvcresponsecode'] = df.apply(lambda x: 3 if x.cvcresponsecode > 2 else x.cvcresponsecode, axis = 1)
df['label'] = df.apply(lambda x: 1 if x.simple_journal == "Chargeback" else 0, axis = 1)
df['creationdate_timestamp'] = df.apply(string_to_timestamp, axis = 1)
df['creationdate'] = df.apply(adjust_creationdate, axis = 1)
df['creationdate'] = pd.to_datetime(df['creationdate'])
print(df.dtypes)
df.head()

txid                                     int64
bookingdate                             object
issuercountrycode                       object
txvariantcode                           object
bin                                    float64
amount                                 float64
currencycode                            object
shoppercountrycode                      object
shopperinteraction                      object
simple_journal                          object
cardverificationcodesupplied            object
cvcresponsecode                          int64
creationdate                    datetime64[ns]
accountcode                             object
mail_id                                  int64
ip_id                                    int64
card_id                                  int64
label                                    int64
creationdate_timestamp                 float64
dtype: object


,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp
0,1,2015-11-09 14:26:51,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09
1,2,2015-11-09 14:27:38,MX,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-02,MexicoAccount,101299,78749,151595,1,1.435805e+09
2,3,2015-11-23 16:34:16,MX,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-02,MexicoAccount,278604,70594,242142,1,1.435840e+09
3,4,2015-11-23 16:34:51,MX,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-03,MexicoAccount,47409,113648,181744,1,1.435903e+09
4,5,2015-11-09 14:26:08,MX,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-08,MexicoAccount,205501,83553,97271,1,1.436373e+09


In [4]:
df_less = df.drop(['txid', 'bookingdate', 'simple_journal'], axis=1)
df_less = df_less.sort_values(by=['creationdate'], ascending=True)
df_less.head()

,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp
0,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,True,0,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09
59886,GB,visadebit,465858.0,3995.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,308365,74702,209050,0,1.435772e+09
59887,GB,visadebit,446291.0,1895.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,274879,269031,22055,0,1.435772e+09
59888,GB,visadebit,476248.0,3695.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,84503,315241,211339,0,1.435772e+09
59889,GB,visadebit,446238.0,9495.0,GBP,GB,Ecommerce,True,1,2015-07-01,UKAccount,56386,48485,141732,0,1.435772e+09


In [5]:
with open('dataframe.pickle', 'wb') as f:
    pickle.dump(df_less, f)

In [6]:
(issuercountry_set, txvariantcode_set, currencycode_set, shoppercountry_set, interaction_set,
verification_set, accountcode_set, mail_id_set, ip_id_set, card_id_set) = [set() for _ in xrange(10)]
(issuercountry_dict, txvariantcode_dict, currencycode_dict, shoppercountry_dict, interaction_dict,
verification_dict, accountcode_dict, mail_id_dict, ip_id_dict, card_id_dict) = [{} for _ in xrange(10)]
issuercountry_set = df_less.issuercountrycode.unique()
txvariantcode_set = df_less.txvariantcode.unique()
currencycode_set = df_less.currencycode.unique()
shoppercountry_set = df_less.shoppercountrycode.unique()
interaction_set = df_less.shopperinteraction.unique()
verification_set = df_less.cardverificationcodesupplied.unique()
accountcode_set = df_less.accountcode.unique()
mail_id_set = df_less.mail_id.unique()
ip_id_set = df_less.ip_id.unique()
card_id_set = df_less.card_id.unique()

In [7]:
with open('feature_sets.pickle', 'wb') as f:
    pickle.dump((issuercountry_set,txvariantcode_set,currencycode_set,shoppercountry_set,interaction_set,verification_set,accountcode_set,mail_id_set,ip_id_set,card_id_set), f)